In [1]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Model, load_model, clone_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Input, Dense, Multiply, Add, Lambda, Concatenate, Reshape, Flatten
from tensorflow.keras.initializers import GlorotUniform, RandomUniform, Constant
from tensorflow.keras.callbacks import LambdaCallback
import jax
import jax.numpy as jnp
from jax import random, grad, jit


In [2]:
# Cargar los módulos preentrenados (unit_module y carry_module)
unit_module = load_model('unit_module.keras')
carry_module = load_model('carry_module.keras')
unit_module.trainable = False
carry_module.trainable = False
unit_module.name = 'unit_model'
carry_module.name = 'carry_model'

In [3]:
# Función para generar los datos
def generate_final_data():
    x_data = []
    y_data = []
    for a_dec in range(10):
        for a_unit in range(10):
            for b_dec in range(10):
                for b_unit in range(10):
                    x_data.append([a_dec, a_unit, b_dec, b_unit])  # Entrada
                    sum_units = (a_unit + b_unit) % 10
                    carry_units = 1 if (a_unit + b_unit) >= 10 else 0
                    sum_dec = (a_dec + b_dec + carry_units) % 10
                    carry_dec = 1 if (a_dec + b_dec + carry_units) >= 10 else 0
                    y_data.append([carry_dec, sum_dec, sum_units])  # Salida
    return jnp.array(x_data), jnp.array(y_data)

# Función para crear parámetros entrenables (v_0, ..., v_35)
def init_params():
    v_values_init = jnp.array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                               0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                               0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0], dtype=jnp.float32)
    key = random.PRNGKey(0)
    keys = random.split(key, 35)
    v_params = {f'v{i}': random.normal(keys[i], (1,)) * epsilon + v_values_init[i] for i in range(35)}
    return v_params

# Función de pérdida
def loss_fn(params, x, y):
    y_pred_1, y_pred_2, y_pred_3 = model(params, x)
    return jnp.mean((y_pred_1 - y[0]) ** 2) + jnp.mean((y_pred_2 - y[1]) ** 2) + jnp.mean((y_pred_3 - y[2]) ** 2)
    
# Función para entrenar el modelo
def update_params(params, x, y, lr):
    # Asegúrate de usar JAX para los gradientes y operaciones
    gradients = grad(loss_fn)(params, x, y)
    step_loss = loss_fn(params, x, y)
    new_params = jax.tree.map(lambda p, g: p - lr * g, params, gradients)
    return new_params, step_loss


def train_model(params, x_train, y_train, lr=0.01, epochs=100):
    final_loss = 0
    # Convertir x_train y y_train a arrays de JAX (si aún no lo son)
    x_train = jnp.array(x_train)
    y_train = jnp.array(y_train)
    
    # Entrenar el modelo
    for epoch in range(epochs):  # Número de épocas
        params, step_loss = update_params(params, x_train[epoch], y_train[epoch], lr)
        final_loss += step_loss
        if epoch % 10 == 0:
            print(f"Epoch {epoch}, Loss: {step_loss}")
        
    final_loss = final_loss / epochs
    return params, final_loss

# Función para imprimir las predicciones y el loss en cada época
def print_predictions_and_loss(epoch, predictions, y_train):
    pred_count = 0
    total_examples = x_train.shape[0]
    
    for i in range(total_examples):
        # Obtener las predicciones para la unidad, decena y acarreo
        normalized_pred = [int(jnp.round(predictions[j][i])) for j in range(3)]
        
        # Concatenar las predicciones en un número de 3 dígitos
        concatenated_pred = int("".join(str(pred) for pred in normalized_pred))
        
        # Generar la salida esperada, concatenando los valores reales de y_train
        expected_output = int("".join(str(int(round(val))) for val in y_train[i]))
        
        # Comprobar si la predicción es igual a la salida esperada
        if concatenated_pred == expected_output:
            pred_count += 1

    print(f"Epoch {epoch + 1}:")
    print(f"Predicciones correctas: {pred_count} de {total_examples}")
    print("-" * 40)

    # Si todas las predicciones son correctas, detener el entrenamiento
    if pred_count == total_examples:
        print("¡Todas las combinaciones han sido aprendidas correctamente! Deteniendo entrenamiento.")
        return True
    return False


def count_predictions(params, x_train, y_train):
    pred_count = 0
    total_examples =  x_train.shape[0]   
    
    for i in range(total_examples):
        prediction = model(params, x_train[i])
        # Obtener las predicciones para la unidad, decena y acarreo
        normalized_pred = [int(jnp.round(prediction[j].item())) for j in range(3)]
        
        if normalized_pred[0] == y_train[i,0]:
            if normalized_pred[1] == y_train[i,1]:
                if normalized_pred[2] == y_train[i,2]:
                    pred_count += 1
                    
    print(f"Predicciones correctas: {pred_count} de {total_examples}")

    if pred_count == total_examples:
        print("¡Todas las combinaciones han sido aprendidas correctamente!")
        

def predictions(params, x_train, y_train):
    pred_count = 0
    total_examples = x_train.shape[0]   
    
    for i in range(total_examples):
        prediction = model(params, x_train[i])
        # Obtener las predicciones para la unidad, decena y acarreo
        normalized_pred = [int(jnp.round(prediction[j].item())) for j in range(3)]
        
        # Concatenar las predicciones en un número de 3 dígitos
        concatenated_pred = int("".join(str(pred) for pred in normalized_pred))
        
        # Generar la salida esperada, concatenando los valores reales de y_train
        expected_output = int("".join(str(int(round(val))) for val in y_train[i]))
        
        # Comprobar si la predicción es igual a la salida esperada
        if concatenated_pred == expected_output:
            pred_count += 1

    print(f"Predicciones correctas: {pred_count} de {total_examples}")

    if pred_count == total_examples:
        print("¡Todas las combinaciones han sido aprendidas correctamente!")

In [4]:
# Modelo dinámico en JAX
def model(params, x):
    # Extraer unidades y decenas de los valores de entrada
    units_input = jnp.array([x[1], x[3]])
    decs_input = jnp.array([x[0], x[2]])
    incorrect_option_1 = jnp.array([x[0], x[1]])
    incorrect_option_2 = jnp.array([x[0], x[3]])
    incorrect_option_3 = jnp.array([x[1], x[2]])
    incorrect_option_4 = jnp.array([x[2], x[3]])
    units_input = units_input[None, None, :]
    decs_input = decs_input[None, None, :]
    incorrect_option_1 = incorrect_option_1[None, None, :]
    incorrect_option_2 = incorrect_option_2[None, None, :]
    incorrect_option_3 = incorrect_option_3[None, None, :]
    incorrect_option_4 = incorrect_option_4[None, None, :]
    
    # Llamar a los modelos unit_module y carry_module
    unit_output = jnp.array(unit_module(units_input))  # Salida para unidades
    carry_output_unit = jnp.array(carry_module(units_input))  # Salida de acarreo de unidades
    dec_output = jnp.array(unit_module(decs_input))  # Salida para decenas
    carry_output_dec = jnp.array(carry_module(decs_input))  # Salida de acarreo de decenas

    incorrect_option_1_output = jnp.array(unit_module(incorrect_option_1)) 
    carry_incorrect_option_1 = jnp.array(carry_module(incorrect_option_1))  
    incorrect_option_2_output = jnp.array(unit_module(incorrect_option_2)) 
    carry_incorrect_option_2 = jnp.array(carry_module(incorrect_option_2))  
    incorrect_option_3_output = jnp.array(unit_module(incorrect_option_3)) 
    carry_incorrect_option_3 = jnp.array(carry_module(incorrect_option_3))  
    incorrect_option_4_output = jnp.array(unit_module(incorrect_option_4)) 
    carry_incorrect_option_4 = jnp.array(carry_module(incorrect_option_4))  

    # Tomar el valor máximo de las predicciones (argmax en JAX)
    unit_val = jnp.argmax(unit_output, axis=-1)
    carry_unit_val = jnp.argmax(carry_output_unit, axis=-1)
    dec_val = jnp.argmax(dec_output, axis=-1)
    carry_dec_val = jnp.argmax(carry_output_dec, axis=-1)
    incorrect_option_1_val = jnp.argmax(incorrect_option_1_output, axis=-1)
    carry_incorrect_option_1_val = jnp.argmax(carry_incorrect_option_1, axis=-1)
    incorrect_option_2_val = jnp.argmax(incorrect_option_2_output, axis=-1)
    carry_incorrect_option_2_val = jnp.argmax(carry_incorrect_option_2, axis=-1)
    incorrect_option_3_val = jnp.argmax(incorrect_option_3_output, axis=-1)
    carry_incorrect_option_3_val = jnp.argmax(carry_incorrect_option_3, axis=-1)
    incorrect_option_4_val = jnp.argmax(incorrect_option_4_output, axis=-1)
    carry_incorrect_option_4_val = jnp.argmax(carry_incorrect_option_4, axis=-1)

    # Calcular las salidas combinadas con los parámetros v
    salida_1 = ((params['v0'] * carry_dec_val) + (params['v1'] * dec_val) + (params['v2'] * carry_unit_val) + (params['v3'] * unit_val)
                + (params['v0'] * carry_incorrect_option_4_val) + (params['v0'] * incorrect_option_4_val) + (params['v0'] * carry_incorrect_option_3_val) + (params['v0'] * incorrect_option_3_val)
                + (params['v0'] * carry_incorrect_option_2_val) + (params['v0'] * incorrect_option_2_val) + (params['v0'] * carry_incorrect_option_1_val) + (params['v0'] * incorrect_option_1_val))
    salida_2 = ((params['v4'] * carry_dec_val) + (params['v5'] * dec_val) + (params['v6'] * carry_unit_val) + (params['v7'] * unit_val)
                + (params['v0'] * carry_incorrect_option_4_val) + (params['v0'] * incorrect_option_4_val) + (params['v0'] * carry_incorrect_option_3_val) + (params['v0'] * incorrect_option_3_val)
                + (params['v0'] * carry_incorrect_option_2_val) + (params['v0'] * incorrect_option_2_val) + (params['v0'] * carry_incorrect_option_1_val) + (params['v0'] * incorrect_option_1_val))
    salida_3 = ((params['v8'] * carry_dec_val) + (params['v9'] * dec_val) + (params['v10'] * carry_unit_val) + (params['v11'] * unit_val)
                + (params['v0'] * carry_incorrect_option_4_val) + (params['v0'] * incorrect_option_4_val) + (params['v0'] * carry_incorrect_option_3_val) + (params['v0'] * incorrect_option_3_val)
                + (params['v0'] * carry_incorrect_option_2_val) + (params['v0'] * incorrect_option_2_val) + (params['v0'] * carry_incorrect_option_1_val) + (params['v0'] * incorrect_option_1_val))

    return salida_1, salida_2, salida_3

In [57]:
x_train, y_train = generate_final_data()
params = init_params()
x=x_train[0]
print(jnp.array([x[1], x[3]]))
new_params, final_loss = train_model(params, x_train, y_train, lr=0.01, epochs=100)
print(final_loss)

# Hacer predicciones después del entrenamiento
predictions = model(params, x_train)
print("Predicciones:", predictions)

AttributeError: Exception encountered when calling Multiply.call().

[1m'ResourceVariable' object has no attribute 'sparse'[0m

Arguments received by Multiply.call():
  • args=(['tf.Tensor(shape=(1,), dtype=float32)', '<KerasTensor shape=(1,), dtype=float32, sparse=False, name=keras_tensor_22>'],)
  • kwargs=<class 'inspect._empty'>

In [ ]:
model = build_dynamic_model()
model.summary()

for var in model.trainable_variables:
    if var.name == "v_values:0":
        print("Valores iniciales de v_values:", var.numpy())


# Predicciones
total_examples = x_train.shape[0]
pred_count = 0

# Realizar las predicciones para todos los ejemplos
predictions = model.predict(x_train)

# Si `predictions` contiene múltiples arrays (uno por salida del modelo):
if isinstance(predictions, list):
    # Concatenamos las predicciones en columnas
    predictions_df = pd.DataFrame(
        {f"Salida_{i+1}": pred.flatten() for i, pred in enumerate(predictions)}
    )
else:
    # Si `predictions` es un solo array
    predictions_df = pd.DataFrame(predictions)

# Guardar como archivo CSV
predictions_df.to_csv("predicciones_completas.csv", index=False)

# Inicializar listas para almacenar las predicciones y las salidas reales
predicted_values = []
expected_values = []

for i in range(total_examples):
    # Normalizar y redondear la predicción (cada salida del modelo)
    normalized_pred = [
        np.round(predictions[0][i]).astype(int),  # Primer valor del primer array
        np.round(predictions[1][i]).astype(int),  # Primer valor del segundo array
        np.round(predictions[2][i]).astype(int)   # Primer valor del tercer array
    ]
    
    # Concatenar los tres elementos en normalized_pred como un solo número
    concatenated_pred = int("".join(str(pred) for pred in normalized_pred))

    # Comparar la predicción con la salida real
    expected_output = int("".join(str(int(round(val))) for val in y_train[i]))  # Convertir la salida esperada en un número
    
    # Almacenar en las listas
    predicted_values.append(concatenated_pred)
    expected_values.append(expected_output)
    
    if concatenated_pred == expected_output:
        pred_count += 1    

print(f'Predicciones correctas: {pred_count} de {total_examples}.')

# Crear un DataFrame con los datos
data = {
    "Predicción": predicted_values,
    "Valor Esperado": expected_values
}
df = pd.DataFrame(data)

# Guardar como archivo CSV
df.to_csv("predicciones.csv", index=False)

print(f"Archivo 'predicciones.csv' guardado con éxito.")